In [1]:
import pyodbc
from datetime import datetime
import os

try: 

    conn = pyodbc.connect(
        "DRIVER={ODBC Driver 17 for SQL Server};"
        "SERVER=YUKTA\\SQLEXPRESS02;"
        "DATABASE=python;"
        "Trusted_Connection=yes;"
    )

    cursor = conn.cursor()
    print("\n== Welcome to Yukta's Restaurant ==")
    print("\n----------- MENU -----------")
    cursor.execute("SELECT item_name, price FROM Menu")
    menu_items = cursor.fetchall()

    for item in menu_items:
        print(f"{item[0]:<20} Rs {item[1]}")

    print("----------------------------\n")



    order_id = int(input("Enter Order ID: "))
    total_bill = 0
    items_list = []



    while True:
        item_name = input("Enter Item Name (or type 'done'): ")

        if item_name.lower() == "done":
            break

        quantity = int(input("Enter Quantity: "))

        
        cursor.execute("SELECT price FROM Menu WHERE item_name = ?", item_name)
        result = cursor.fetchone()

        if result:
            price = result[0]
            total = price * quantity
            total_bill += total

            now = datetime.now()

            # Insert into Orders table
            cursor.execute("""
                INSERT INTO Orders (order_id, item_name, price, quantity, total, order_date, order_time)
                VALUES (?, ?, ?, ?, ?, ?, ?)
            """, order_id, item_name, price, quantity, total,
                now.date(), now.time())

            conn.commit()

            items_list.append((item_name, price, quantity, total))

        else:
            print("Item not found in menu!")

    #Printing bill
    now = datetime.now()
    datestamp = now.strftime("%d-%m-%Y")
    timestamp = now.strftime("%I:%M %p")

    bill= ""
    bill+= "="*50 + "\n"
    bill+= "{:^50}\n".format("Yukta's Restaurant")
    bill+= "="*50 + "\n"
    bill+= f"{datestamp:<25}{timestamp:>25}\n"
    bill+= "-"*50 + "\n"
    bill+= "{:<15}{:<10}{:<10}{:<10}\n".format("Item", "Price", "Qty", "Total")
    bill+= "-"*50 + "\n"

    for item in items_list:
        bill += "{:<15}{:<10}{:<10}{:<10}\n".format(item[0], item[1], item[2], item[3])

    bill+= "-"*50 + "\n"
    bill+= "{:<35}{:<10}\n".format("Grand Total:", total_bill)
    bill+= "="*50 + "\n"

    print("\n", bill)


    #For saving bill
    path = r"C:\Users\barad\OneDrive\Desktop\New folder (2)"
    file_name = f"Bill_{order_id}.txt"
    full_path = os.path.join(path, file_name)

    with open(full_path, "w") as file:
        file.write(bill)

    print("Bill saved at:", full_path)

except pyodbc.Error as db_error:
    print("Database connection failed:", db_error)

except Exception as e:
    print("Unexpected error occurred:", e)
    
    conn.close()


== Welcome to Yukta's Restaurant ==

----------- MENU -----------
Pizza                Rs 250
Burger               Rs 150
Pasta                Rs 200
Coffee               Rs 100
Sandwich             Rs 120
French Fries         Rs 130
Cold Drink           Rs 80
Tea                  Rs 50
Ice Cream            Rs 90
Momos                Rs 140
----------------------------


                Yukta's Restaurant                
18-02-2026                                01:49 PM
--------------------------------------------------
Item           Price     Qty       Total     
--------------------------------------------------
Pizza          250       10        2500      
--------------------------------------------------
Grand Total:                       2500      

Bill saved at: C:\Users\barad\OneDrive\Desktop\New folder (2)\Bill_13.txt
